In [1]:
# importing necessary libraries
import numpy as np 
import os
import random
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# saving list of names of txt files
directory_path = r'C:\Users\rohit\Python\Personal Projects\shaayar\ghazal_data\all'

txt_files = [file for file in os.listdir(directory_path) if file.endswith('.txt')]

In [3]:
# stroing all poems as a single string
poems_data = ""

for txt_file in txt_files:
    file_path = os.path.join(directory_path, txt_file)
    
    with open(file_path, 'r', encoding='utf-8') as file:
        poem_content = file.read()
    
    poems_data += poem_content

In [4]:
print(poems_data[:1000])

gulon ko sunna zara tum sadaen bheji hain
gulon ke haath bahut si duaen bheji hain

jo aftab kabhi bhi ghurub hota nahin
hamara dil hai usi ki shuaen bheji hain

agar jalae tumhen bhi shifa mile shayad
ik aise dard ki tum ko shuaen bheji hain

tumhari khushk si ankhen bhali nahin lagtin
vo saari chizen jo tum ko rulaen, bheji hain

siyah rang chamakti hui kanari hai
pahan lo achchhi lagengi ghaTaen bheji hain

tumhare khvab se har shab lipaT ke sote hain
sazaen bhej do ham ne khataen bheji hain

akela patta hava men bahut buland uḌa
zamin se paanv uThao havaen bheji hain
koi hua na ru-kash Tak meri chashm-e-tar se
kya kya na abr aa kar yaan zor zor barse

vahshat se meri yaaro khatir na jama rakhiyo
phir aave ya na aave nau pur uTha jo ghar se

ab juun sarishk un se phirne ki chashm mat rakh
jo khaak men mile hain gir kar tiri nazar se

didar khvah us ke kam hon to shor kam ho
har subh ik qayamat uThti hai us ke dar se

daagh ek ho jila bhi khuun ek ho baha bhi
ab bahs kya hai dil se k

In [5]:
# getting list of characters used in the poems and vocabulary size
chars = sorted(list(set(poems_data)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !',-/:;ABCDEFGHIJKLMNOPQRSTUVWYZabcdefghijklmnopqrstuvwxyz،؟ءآؤئابتثجحخدذرزسشصضطظعغفقلمنويًٹپچڈڑکگںھہیےḌ‘’”
109


In [6]:
# creating function for string to integer and integer to string
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("kya baat hai!"))
print(decode(encode("kya baat hai!")))

[44, 58, 34, 1, 35, 34, 34, 53, 1, 41, 34, 42, 2]
kya baat hai!


In [7]:
# encoding the data
data = torch.tensor(encode(poems_data), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:500])

torch.Size([2005787]) torch.int64
tensor([40, 54, 45, 48, 47,  1, 44, 48,  1, 52, 54, 47, 47, 34,  1, 59, 34, 51,
        34,  1, 53, 54, 46,  1, 52, 34, 37, 34, 38, 47,  1, 35, 41, 38, 43, 42,
         1, 41, 34, 42, 47,  0, 40, 54, 45, 48, 47,  1, 44, 38,  1, 41, 34, 34,
        53, 41,  1, 35, 34, 41, 54, 53,  1, 52, 42,  1, 37, 54, 34, 38, 47,  1,
        35, 41, 38, 43, 42,  1, 41, 34, 42, 47,  0,  0, 43, 48,  1, 34, 39, 53,
        34, 35,  1, 44, 34, 35, 41, 42,  1, 35, 41, 42,  1, 40, 41, 54, 51, 54,
        35,  1, 41, 48, 53, 34,  1, 47, 34, 41, 42, 47,  0, 41, 34, 46, 34, 51,
        34,  1, 37, 42, 45,  1, 41, 34, 42,  1, 54, 52, 42,  1, 44, 42,  1, 52,
        41, 54, 34, 38, 47,  1, 35, 41, 38, 43, 42,  1, 41, 34, 42, 47,  0,  0,
        34, 40, 34, 51,  1, 43, 34, 45, 34, 38,  1, 53, 54, 46, 41, 38, 47,  1,
        35, 41, 42,  1, 52, 41, 42, 39, 34,  1, 46, 42, 45, 38,  1, 52, 41, 34,
        58, 34, 37,  0, 42, 44,  1, 34, 42, 52, 38,  1, 37, 34, 51, 37,  1, 44,
      

In [9]:
# splitting data in train and test set
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

In [10]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [11]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [12]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [13]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [14]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [15]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [18]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [19]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)

0.215405 M parameters
step 0: train loss 4.7938, val loss 4.7940
step 100: train loss 2.4896, val loss 2.4825
step 200: train loss 2.4020, val loss 2.3671
step 300: train loss 2.3160, val loss 2.2922
step 400: train loss 2.2468, val loss 2.2394
step 500: train loss 2.1866, val loss 2.1808
step 600: train loss 2.1428, val loss 2.1300
step 700: train loss 2.1188, val loss 2.1045
step 800: train loss 2.0987, val loss 2.0827
step 900: train loss 2.0562, val loss 2.0508
step 1000: train loss 2.0361, val loss 2.0118
step 1100: train loss 2.0095, val loss 2.0001
step 1200: train loss 1.9789, val loss 1.9696
step 1300: train loss 1.9784, val loss 1.9583
step 1400: train loss 1.9411, val loss 1.9272
step 1500: train loss 1.9247, val loss 1.9168
step 1600: train loss 1.9228, val loss 1.9059
step 1700: train loss 1.9060, val loss 1.8961
step 1800: train loss 1.8809, val loss 1.8776
step 1900: train loss 1.8803, val loss 1.8746
step 2000: train loss 1.8719, val loss 1.8670
step 2100: train loss 1.

In [20]:
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))



main mahki mire ghar kho-sumudana mira
dor aah se rahi sardi na hi ki
khayana kya haisi ka ab diyaiz

sakgane ab dil hai ave chalte kya gaya
saar mehr-e-nag khate aur oflanda men khud ko
ye marna jaana gubar hai ham gae

Aasarad koi baaya nakse pahla hai
kanten hain ditange ab ki
kareng marg maud nigahon ka aang lagne

khuub hi rubhai ho gayya jiye shahi vala
kahan and ke liye jo lekin ke larg-e-khaba-og
nazzar vo khayyabon se rahin

my gulenten dind mohlegi salvanm bas chalakte aa koi dhail gaya

cham-e-vahi kaam hai 'zamir' ka tasf

magar jabiye mangas hai diya kabhi
shauha-e-roz sakhi-Ḍanga magar saad ne kis be-ham
mulfas bhaar ho yaan
tha challenga aaya main dega
bacha toḌe agai nahin but de ke

hai base hain kabs rakta hairam jaenan tha
itne khiran hoge sio tamavar like jaaen
koi dekha na ko chukte Tahin banan ke lahkita bazhaton kahan aae Dhalaega
'mabazai tha kyuun kya nikal ye duaa main
kya koise baaaq hai mai-qaya se qadim men ik magar
masagaḌo hai jan-س-bazzar mare Thog de 